# Projeto 1 - Ciência dos Dados

Nome: Lucas Franco Florentino

Nome: Lucas Kang

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import nltk
from nltk.corpus import stopwords

Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [2]:
filename = 'kit kat.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo kit kat.xlsx, tudo certo para prosseguir com a prova!


In [3]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,class
0,@biaturci bia tmb é didática!!!,0
1,"6 kit kat por 10 pila na americanas, vou logo ...",2
2,e se eu trocar meu nick pra: kit kat...\nalgue...,0
3,@pivetabeca o preço q eh uma barra de kit kat ...,1
4,qria um ovo d chocolate da alpino ou da kit ka...,4


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,class,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,queria dar de presente pra minha irmã o kit ka...,1,NaN,NaN,NaN,NaN,NaN
1,@123luigig @lilyandgia kit kat é mt bom,4,NaN,NaN,NaN,NaN,NaN
2,kit kat de morango e de limão são tudo pra mim 🥺🤤,4,NaN,NaN,NaN,NaN,NaN
3,tudo que eu precisava neste exato momento era:...,4,NaN,NaN,NaN,NaN,NaN
4,kit kat eu te amo esse tuíte é pra vc,4,NaN,NaN,NaN,NaN,NaN


## Classificador automático de sentimento

Para a classificação foi usada 5 tipos, Muito relevante, Relevante, Neutro, Irrelevantee Muito irrelevante. A classificação foi baseada no nível de relação que o tweet tinha com nosso produto. Portanto, um tweet Muito relevante ou Relevante, deveria apresentar uma opnião explicita sobre o produto seja ela positiva ou negativa. Um tweet Neutro, seria aquele que somente cita o nome do produto. E para finalizar um Muito irrelevante ou Irrelevante deveria não citar o nome do produto. 

### Montando um Classificador Naive-Bayes

Para iniciar a progamação do classificador, foi necessário limpar os dados para eliminar possíveis distratores no processo de classificação. Desse modo, a função 'cleanup' junto com a biblioteca nltk, realizaram a limpeza da base de dados.
Após a limpeza, os dados são separados em listas e feita a contagem das frequencias absolutas e relativas de cada lista.

In [5]:
# Faz a limpeza da base de dados e retira stop words
import re 
def cleanup(text):
    punctuation = '[/\@!-.:?;\n""“”,_]' 
    pattern = re.compile(punctuation)
    texto_limpo= re.sub(pattern, '', text)
    return  texto_limpo

nltk.download('stopwords')
stop = stopwords.words('portuguese')
stop_list = ["de", "é", "á", "à", "ao", "a", "o", "é", "rt", '"', "“", "'", ",", "(", ")", "$", "%", "*", "&", "+", "="]
stop.extend(stop_list)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
#Define as classificações como categorias

train = pd.read_excel(filename)
treino_mrel = train.loc[train['class'] == 4,:]
treino_rel = train.loc[train['class'] == 3,:]
treino_neut = train.loc[train['class'] == 2,:]
treino_irrel = train.loc[train['class'] == 1,:]
treino_mirrel = train.loc[train['class'] == 0,:]

teste_mrel= test.loc[train['class'] == 4,:]
teste_rel = test.loc[train['class'] == 3,:]
teste_neut = test.loc[train['class'] == 2,:]
teste_irrel = test.loc[train['class'] == 1,:]
teste_mirrel = test.loc[train['class'] == 0,:]

In [7]:
#Separando a base dados para Muito relevante, Relevante, Neutro, Irrelevante, Muito irrelevante
treino_mrel['Treinamento'] = treino_mrel['Treinamento'].astype(str)
treino_rel['Treinamento'] = treino_rel['Treinamento'].astype(str)
treino_neut['Treinamento'] = treino_neut['Treinamento'].astype(str)
treino_irrel['Treinamento'] = treino_irrel['Treinamento'].astype(str)
treino_mirrel['Treinamento'] = treino_mirrel['Treinamento'].astype(str)
 
#Coloca tudo em uma lista
lista_tr_mrel= treino_mrel['Treinamento'].tolist()
lista_tr_rel= treino_rel['Treinamento'].tolist()
lista_tr_neut= treino_neut['Treinamento'].tolist()
lista_tr_irrel= treino_irrel['Treinamento'].tolist()
lista_tr_mirrel= treino_mirrel['Treinamento'].tolist()

<ipython-input-7-90aeb146183d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treino_mrel['Treinamento'] = treino_mrel['Treinamento'].astype(str)
<ipython-input-7-90aeb146183d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treino_rel['Treinamento'] = treino_rel['Treinamento'].astype(str)
<ipython-input-7-90aeb146183d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [8]:
#Função para aplicar a limpeza nos dados de treinamento para:

#Muito Relevante
tr_mrel_limpo = []
for txt in lista_tr_mrel:
    txt = cleanup(txt)
    tr_mrel_limpo.append(txt)
tr_mrel_limpo =[' '.join([w for w in x.lower().split() if w not in stop]) #Aplica a função que retira stop words
    for x in tr_mrel_limpo]
    
tr_mrel_div = []
for txt in tr_mrel_limpo:
    txt= txt.split()
    tr_mrel_div.append(txt)

    
#Relevante

tr_rel_limpo = []
for txt in lista_tr_rel:
    txt = cleanup(txt)
    tr_rel_limpo.append(txt)
tr_rel_limpo = [' '.join([w for w in x.lower().split() if w not in stop]) 
    for x in tr_rel_limpo]
    
tr_rel_div = []
for txt in tr_rel_limpo:
    txt= txt.split()
    tr_rel_div.append(txt)

    
# Neutro

tr_neut_limpo = []
for txt in lista_tr_neut:
    txt = cleanup(txt)
    tr_neut_limpo.append(txt)
tr_neut_limpo = [' '.join([w for w in x.lower().split() if w not in stop]) 
    for x in tr_neut_limpo]
    
tr_neut_div = []
for txt in tr_neut_limpo:
    txt= txt.split()
    tr_neut_div.append(txt)
    
    
#Irrelevante

tr_irrel_limpo = []
for txt in lista_tr_irrel:
    txt = cleanup(txt)
    tr_irrel_limpo.append(txt)
tr_irrel_limpo = [' '.join([w for w in x.lower().split() if w not in stop]) 
    for x in tr_irrel_limpo]
    
tr_irrel_div = []
for txt in tr_irrel_limpo:
    txt= txt.split()
    tr_irrel_div.append(txt)

#Muito irrelevante

tr_mirrel_limpo = []
for txt in lista_tr_mirrel:
    txt = cleanup(txt)
    tr_mirrel_limpo.append(txt)
tr_mirrel_limpo = [' '.join([w for w in x.lower().split() if w not in stop]) 
    for x in tr_mirrel_limpo]
    
tr_mirrel_div = []
for txt in tr_mirrel_limpo:
    txt= txt.split()
    tr_mirrel_div.append(txt)


In [9]:
#Coloca todos os tweets da base de teste em uma lista e realiza a limpeza e a retirada dos stopwords 
test.loc[:,'Teste'] = test.loc[:,'Teste'].astype(str)
teste_limpo = []
for txt in test['Teste'].tolist():
    txt = cleanup(txt)
    teste_limpo.append(txt)
    teste_limpo = [' '.join([w for w in x.lower().split() if w not in stop]) 
    for x in teste_limpo]
div_teste = []
for txt in teste_limpo:
    txt = txt.split()
    div_teste.append(txt)
print(div_teste)   


[['queria', 'dar', 'presente', 'pra', 'irmã', 'kit', 'kat', 'parceria', 'now', 'united', '99', 'reais', 'chocolate', '🤡', 'cara', 'tentando', 'cancelar', 'compra', 'email', 'ver', 'valor', '135', 'estalecas', 'frete', 'sabendo', 'deixaram', 'claro', 'antes', 'compra', 'httpstcoeverznihri'], ['123luigig', 'lilyandgia', 'kit', 'kat', 'mt', 'bom'], ['kit', 'kat', 'morango', 'limão', 'tudo', 'pra', 'mim', '🥺🤤'], ['tudo', 'precisava', 'neste', 'exato', 'momento', 'eraum', 'bolo', 'recheio', 'abacaxium', 'bolo', 'churrosum', 'bolo', 'algum', 'chantily', 'gostoso', 'cremoso', 'cima', 'recheio', 'chocolate', 'doce', 'leiteuma', 'torta', 'morangos', 'kit', 'kat', 'qlqr', 'fazia', 'feliz'], ['kit', 'kat', 'amo', 'tuíte', 'pra', 'vc'], ['crystaltriste', '3', 'anos', 'info', 'sabe', 'hackear', 'links', 'achar', 'links'], ['liviabarroos', 'fala', 'escuto', 'grito', 'causa', 'kit', 'kat'], ['professora', 'disse', 'ligar', 'camera', 'ganha', 'kit', 'kat', 'httpstcoa2ywkrlzx7'], ['esperei', 'tanto', '

## Iniciando a separação das palavras em listas e calculando suas frequências:

In [10]:
#Contagem da frequencia das palavras dos tweets muito relevantes
mrel_palavras= []
for txt in tr_mrel_div:
    for wrd in txt:
        mrel_palavras.append(wrd )
mrel_palavras_serie = pd.Series(mrel_palavras)
#mrel_palavras_serie 


In [11]:
#Contagem da frequencia das palavras dos tweets relevantes
rel_palavras= []
for txt in tr_rel_div:
    for wrd in txt:
        rel_palavras.append(wrd)
rel_palavras_serie = pd.Series(rel_palavras)
#rel_palavras_serie


In [12]:
#Contagem da frequencia das palavras dos tweets neutros
neut_palavras= []
for txt in tr_neut_div:
    for wrd in txt:
        neut_palavras.append(wrd)
neut_palavras_serie = pd.Series(neut_palavras)
#neut_palavras_serie


In [13]:
#Contagem da frequencia das palavras dos tweets irrelevantes
irrel_palavras= []
for txt in tr_irrel_div:
    for wrd in txt:
        irrel_palavras.append(wrd)
irrel_palavras_serie = pd.Series(irrel_palavras)
#irrel_palavras_serie


In [14]:
#Contagem da frequencia das palavras dos tweets muito irrelevantes
mirrel_palavras= []
for txt in tr_mirrel_div:
    for wrd in txt:
        mirrel_palavras.append(wrd)
mirrel_palavras_serie = pd.Series(mirrel_palavras)
#mtirrel_palavras_serie


In [15]:
#Contagem das frequencias das palavras para Muito relevante:

# Frequência absoluta
freq_abs_mrel = mrel_palavras_serie.value_counts()
print("\nFrequencias absolutas dos tweets muito relevantes: \n", freq_abs_mrel)

# Frequência relativa
freq_rela_mrel = mrel_palavras_serie.value_counts(normalize = True)
print("\nFrequencias relativas: \n", freq_rela_mrel)



Frequencias absolutas dos tweets muito relevantes: 
 kat           6
kit           6
bom           2
vamos         1
jvlaruccia    1
fodase        1
comendo       1
ovo           1
dms           1
branco        1
hm            1
acho          1
belo          1
d             1
qria          1
alpino        1
mar3azinha    1
agorinhaa     1
chocolate     1
perfo         1
sorvete       1
alvxaro       1
eh            1
sinceros      1
prefiro       1
ser           1
mto           1
gostoso       1
nenhum        1
dtype: int64

Frequencias relativas: 
 kat           0.150
kit           0.150
bom           0.050
vamos         0.025
jvlaruccia    0.025
fodase        0.025
comendo       0.025
ovo           0.025
dms           0.025
branco        0.025
hm            0.025
acho          0.025
belo          0.025
d             0.025
qria          0.025
alpino        0.025
mar3azinha    0.025
agorinhaa     0.025
chocolate     0.025
perfo         0.025
sorvete       0.025
alvxaro       0.025
eh 

In [16]:
# Contagem das frequencias das palavras para Relevante:

# Frequência absoluta
freq_abs_rel = rel_palavras_serie.value_counts()
print("\nFrequencias absolutas dos tweets relevantes: \n", freq_abs_rel)


#Frequência relativa
freq_rela_rel = rel_palavras_serie.value_counts(normalize = True)
print("\nFrequencias relativas: \n", freq_rela_rel)


Frequencias absolutas dos tweets relevantes: 
 kit         52
kat         52
limão        8
melhor       7
favorito     6
            ..
demais       1
bfrcarou     1
mt           1
bolo         1
descobri     1
Length: 213, dtype: int64

Frequencias relativas: 
 kit         0.133333
kat         0.133333
limão       0.020513
melhor      0.017949
favorito    0.015385
              ...   
demais      0.002564
bfrcarou    0.002564
mt          0.002564
bolo        0.002564
descobri    0.002564
Length: 213, dtype: float64


In [17]:
# Contagem das frequencias das palavras para Neutro:

# Frequência absoluta
freq_abs_neut = neut_palavras_serie.value_counts()
print("\nFrequencias absolutas dos tweets neutros: \n", freq_abs_neut)

# Frequência relativa
freq_rela_neut = neut_palavras_serie.value_counts(normalize = True)
print("\nFrequencias relativas: \n", freq_rela_neut)


Frequencias absolutas dos tweets neutros: 
 kat             38
kit             38
pra             10
vou              7
comer            6
                ..
ngm              1
diz              1
🙆💓               1
pois             1
thaliaraujo7     1
Length: 225, dtype: int64

Frequencias relativas: 
 kat             0.106443
kit             0.106443
pra             0.028011
vou             0.019608
comer           0.016807
                  ...   
ngm             0.002801
diz             0.002801
🙆💓              0.002801
pois            0.002801
thaliaraujo7    0.002801
Length: 225, dtype: float64


In [18]:
# Contagem das frequencias das palavras para Irrelevante:

# Frequência absoluta
freq_abs_irrel = irrel_palavras_serie.value_counts()
print("\nFrequencias absolutas dos tweets irrelevantes: \n", freq_abs_irrel)

# Frequência relativa 
freq_rela_irrel = irrel_palavras_serie.value_counts(normalize = True)
print("\nFrequencias relativas: \n", freq_rela_irrel)


Frequencias absolutas dos tweets irrelevantes: 
 kit          44
kat          43
q             7
pra           7
comi          6
             ..
come          1
únicas        1
dinheiro      1
noah          1
comprando     1
Length: 338, dtype: int64

Frequencias relativas: 
 kit          0.083176
kat          0.081285
q            0.013233
pra          0.013233
comi         0.011342
               ...   
come         0.001890
únicas       0.001890
dinheiro     0.001890
noah         0.001890
comprando    0.001890
Length: 338, dtype: float64


In [19]:
# Contagem das frequencias das palavras para Irrelevante:

# Frequência absoluta
freq_abs_mirrel = mirrel_palavras_serie.value_counts()
print("\nFrequencias absolutas dos tweets muito irrelevantes: \n", freq_abs_mirrel)

# Frequência relativa 
freq_rela_mirrel = mirrel_palavras_serie.value_counts(normalize = True)
print("\nFrequencias relativas: \n", freq_rela_mirrel)


Frequencias absolutas dos tweets muito irrelevantes: 
 kit                   95
kat                   90
pra                   27
ovo                   20
kitkatoff             16
                      ..
podemme                1
consigam               1
subir                  1
httpstcofwzgayvdlw     1
nate                   1
Length: 1046, dtype: int64

Frequencias relativas: 
 kit                   0.054472
kat                   0.051606
pra                   0.015482
ovo                   0.011468
kitkatoff             0.009174
                        ...   
podemme               0.000573
consigam              0.000573
subir                 0.000573
httpstcofwzgayvdlw    0.000573
nate                  0.000573
Length: 1046, dtype: float64


### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

Desse modo, a probabilidade de um tweet ser relevante ou não será baseado em um comparação deste tweet com todas as palavras de nossa base de dados. Isto será feito com um lista de todas as palavras dos tweets que obtivemos, excluindo as repetidas, que serão consideradas como um item de série.

In [20]:
#Cria uma lista com todas as palavras classificadas

total = tr_mrel_div + tr_rel_div + tr_neut_div + tr_irrel_div + tr_mirrel_div
rel_total= []
for txt in total:
    for wrd in txt:
        rel_total.append(wrd)
total_rel_serie = pd.Series(rel_total)
relativo_total = total_rel_serie.value_counts(True)
relativo_total

kit          0.076797
kat          0.074837
pra          0.015686
ovo          0.009804
chocolate    0.007843
               ...   
passe        0.000327
bolinha      0.000327
consigam     0.000327
podemme      0.000327
pular        0.000327
Length: 1474, dtype: float64

Para aumentar a precisão e acurácia do classificador, decidimos em implementar a suavização de Laplace, que elimina as probabilidades de valor zero.

In [21]:
resp = []
 
mrel_prob=len(freq_abs_mrel)/len(relativo_total)
rel_prob=len(freq_abs_rel)/len(relativo_total)
neut_prob=len(freq_abs_neut)/len(relativo_total)
irrel_prob=len(freq_abs_irrel)/len(relativo_total)
mirrel_prob=len(freq_abs_mirrel)/len(relativo_total)
for i in range(len(div_teste)):
    mrel_prob_txt=[]
    rel_prob_txt=[]
    neut_prob_txt=[]
    irrel_prob_txt=[]
    mirrel_prob_txt=[]
    for wrd in div_teste[i]:
        mrel_prob = 1
        rel_prob = 1
        neut_prob = 1
        irrel_prob = 1
        mirrel_prob = 1
        #print('\npalavra: ',palavra)
        mrel_prob *= 1/(len(freq_abs_mrel)+len(relativo_total))
        rel_prob *= 1/(len(freq_abs_rel)+len(relativo_total))
        neut_prob *= 1/(len(freq_abs_neut)+len(relativo_total))
        irrel_prob *= 1/(len(freq_abs_irrel)+len(relativo_total))
        mirrel_prob *=1/(len(freq_abs_mirrel)+len(relativo_total))
 
        if wrd in freq_abs_mrel:
            mrel_prob += (freq_abs_mrel[wrd ])/(len(freq_abs_mrel)+len(relativo_total))
 
        if wrd in freq_abs_rel:
            rel_prob += (freq_abs_rel[wrd])/(len(freq_abs_rel)+len(relativo_total))
 
        if wrd in freq_abs_neut:
            neut_prob += (freq_abs_neut[wrd ])/(len(freq_abs_neut)+len(relativo_total))
 
        if wrd in freq_abs_irrel:
            irrel_prob += (freq_abs_irrel[wrd ])/(len(freq_abs_irrel)+len(relativo_total))
 
        if wrd in freq_abs_mirrel:
            mirrel_prob += (freq_abs_mirrel[wrd ])/(len(freq_abs_mirrel)+len(relativo_total))
 
        mrel_prob_txt.append(mrel_prob)
        rel_prob_txt.append(rel_prob)
        neut_prob_txt.append(neut_prob)
        irrel_prob_txt.append(irrel_prob)
        mirrel_prob_txt.append(mirrel_prob)
        #print('1: ', probFrasedado1)
        #print('0: ', probFrasedado0)
    if np.prod(mrel_prob_txt)*mrel_prob > np.prod(mirrel_prob_txt)*mirrel_prob:
        resp.append(4)
    elif np.prod(rel_prob_txt)*rel_prob > np.prod(irrel_prob_txt)*irrel_prob:
        resp.append(3)
    #elif np.prod(neut_prob_txt)*neut_prob == np.prod(neut_prob_txt)*neut_prob:
        #resp.append(2)
    elif np.prod(irrel_prob_txt)*irrel_prob > np.prod(rel_prob_txt)*rel_prob:
        resp.append(1)
    elif np.prod(mirrel_prob_txt)*mirrel_prob > np.prod(mrel_prob_txt)*mrel_prob:
        resp.append(0)
    else:
        resp.append(2)
        
 
#print(res)
test['Resultado'] = resp
print(resp)


[3, 3, 3, 3, 3, 4, 3, 3, 1, 3, 3, 1, 1, 3, 3, 3, 1, 4, 3, 1, 3, 1, 3, 4, 3, 3, 1, 4, 3, 3, 3, 3, 1, 3, 3, 3, 1, 1, 3, 1, 4, 1, 1, 3, 3, 1, 1, 3, 3, 3, 4, 3, 3, 3, 1, 3, 4, 1, 1, 3, 3, 1, 3, 1, 3, 3, 1, 1, 3, 4, 1, 1, 1, 3, 1, 3, 1, 3, 1, 3, 3, 1, 3, 3, 1, 1, 4, 4, 3, 3, 3, 1, 1, 3, 3, 1, 3, 3, 3, 3, 3, 3, 1, 3, 3, 1, 4, 3, 3, 3, 3, 3, 4, 3, 1, 3, 4, 1, 4, 1, 1, 3, 3, 1, 3, 1, 1, 3, 1, 3, 1, 4, 3, 4, 3, 1, 1, 3, 3, 3, 1, 1, 3, 1, 1, 4, 3, 1, 3, 3, 1, 1, 1, 3, 1, 3, 1, 3, 3, 1, 3, 3, 3, 3, 1, 3, 4, 1, 1, 1, 1, 1, 3, 4, 1, 3, 3]


In [22]:
test['Resultado'] = resp
print(resp)


[3, 3, 3, 3, 3, 4, 3, 3, 1, 3, 3, 1, 1, 3, 3, 3, 1, 4, 3, 1, 3, 1, 3, 4, 3, 3, 1, 4, 3, 3, 3, 3, 1, 3, 3, 3, 1, 1, 3, 1, 4, 1, 1, 3, 3, 1, 1, 3, 3, 3, 4, 3, 3, 3, 1, 3, 4, 1, 1, 3, 3, 1, 3, 1, 3, 3, 1, 1, 3, 4, 1, 1, 1, 3, 1, 3, 1, 3, 1, 3, 3, 1, 3, 3, 1, 1, 4, 4, 3, 3, 3, 1, 1, 3, 3, 1, 3, 3, 3, 3, 3, 3, 1, 3, 3, 1, 4, 3, 3, 3, 3, 3, 4, 3, 1, 3, 4, 1, 4, 1, 1, 3, 3, 1, 3, 1, 1, 3, 1, 3, 1, 4, 3, 4, 3, 1, 1, 3, 3, 3, 1, 1, 3, 1, 1, 4, 3, 1, 3, 3, 1, 1, 1, 3, 1, 3, 1, 3, 3, 1, 3, 3, 3, 3, 1, 3, 4, 1, 1, 1, 1, 1, 3, 4, 1, 3, 3]


In [23]:
test['Resultado'] = resp
test


,Teste,class,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Resultado
0,queria dar de presente pra minha irmã o kit ka...,1,NaN,NaN,NaN,NaN,NaN,3
1,@123luigig @lilyandgia kit kat é mt bom,4,NaN,NaN,NaN,NaN,NaN,3
2,kit kat de morango e de limão são tudo pra mim 🥺🤤,4,NaN,NaN,NaN,NaN,NaN,3
3,tudo que eu precisava neste exato momento era:...,4,NaN,NaN,NaN,NaN,NaN,3
4,kit kat eu te amo esse tuíte é pra vc,4,NaN,NaN,NaN,NaN,NaN,3
...,...,...,...,...,...,...,...,...
172,"kit kat indo queimar no inferno, q delicia de ...",0,NaN,NaN,NaN,NaN,NaN,3
173,rt @df_porto: seguinte:\n\npau no cu dessa mat...,0,NaN,NaN,NaN,NaN,NaN,4
174,@thalesvictorsan tá tudo caro e outra abril tá...,0,NaN,NaN,NaN,NaN,NaN,1
175,vontade de comer um kit kat 🥴,4,NaN,NaN,NaN,NaN,NaN,3


## Performance
 
 A performance do classificador será verificada através da análise de verdadeiros positivos, verdadeiros negativos, falsos positivos e falsos negativos. 

In [24]:
pd.crosstab(test['class'],test['Resultado'])

Resultado,1,3,4
class,,,
0,17,20,18
1,4,10,0
2,18,5,0
3,16,18,1
4,10,40,0


In [25]:
pd.crosstab(test['class'],test['Resultado'],normalize='index')

Resultado,1,3,4
class,,,
0,0.309091,0.363636,0.327273
1,0.285714,0.714286,0.000000
2,0.782609,0.217391,0.000000
3,0.457143,0.514286,0.028571
4,0.200000,0.800000,0.000000


In [27]:
lista_classificacao = test['class'].tolist()
lista_resultado = test['Resultado'].tolist()
 
test_mrelevante = test.loc[test['class'] == 4,:]
test_relevante = test.loc[test['class'] == 3,:]
test_neutro = test.loc[test['class'] == 2,:]
test_irrelevante = test.loc[test['class'] == 1,:]
test_mirrelevante = test.loc[test['class'] == 0,:]
acertos = 0
for i in range(len(lista_classificacao)):
    if lista_classificacao[i] == lista_resultado[i]:
        acertos += 1
porcentagem_acertos = acertos/len(lista_classificacao)
print("Número de acertos: ", acertos)
print("Porcentagem de acertos: {0:.3f}%".format(porcentagem_acertos*100))

Número de acertos:  22
Porcentagem de acertos: 12.429%


In [29]:
lista_classificacao = test['class'].tolist()
lista_resultado  = test['Resultado'].tolist()

verdadeiros_positivos = 0
for i in range(len(lista_classificacao)):
    if lista_classificacao[i] == 4 and lista_resultado[i] == 4:
        verdadeiros_positivos += 1
    elif lista_classificacao[i] == 3 and lista_resultado[i]== 3:
        verdadeiros_positivos += 1
porcentagem_verdadeiros_positivos = verdadeiros_positivos/(len(test_mrelevante)+len(test_relevante))
print("Número de verdadeiros positivos: ", verdadeiros_positivos)
print("Porcentagem de verdadeiros positivos: {0:.3f}%".format(porcentagem_verdadeiros_positivos*100))

Número de verdadeiros positivos:  18
Porcentagem de verdadeiros positivos: 21.176%


In [30]:
falsos_positivos = 0
for i in range(len(lista_classificacao)):
    if lista_classificacao[i] == 0 and lista_resultado[i] == 4:
        falsos_positivos += 1
    elif lista_classificacao[i] == 0 and lista_resultado[i]== 3:
        falsos_positivos += 1
    elif lista_classificacao[i] == 1 and lista_resultado[i]== 4:
        falsos_positivos += 1
    elif lista_classificacao[i] == 1 and lista_resultado[i]== 3:
        falsos_positivos += 1
    elif lista_classificacao[i] == 2 and lista_resultado[i]== 4:
        falsos_positivos += 1
    elif lista_classificacao[i] == 2 and lista_resultado[i]== 3:
        falsos_positivos += 1
porcentagem_falsos_positivos = falsos_positivos/(len(test_mirrelevante)+ len(test_irrelevante))
print("Número de falsos positivos: ", falsos_positivos)
print("Porcentagem de falsos positivos: {0:.3f}%".format(porcentagem_falsos_positivos*100))


Número de falsos positivos:  53
Porcentagem de falsos positivos: 76.812%


In [31]:
verdadeiros_negativos = 0
for i in range(len(lista_classificacao)):
    if lista_classificacao[i] == 0 and lista_resultado[i] == 0:
        verdadeiros_negativos += 1
    elif lista_classificacao[i] == 1 and lista_resultado[i]== 1:
        verdadeiros_negativos += 1
porcentagem_verdadeiros_negativos = verdadeiros_negativos/(len(test_irrelevante) + len(test_mirrelevante))
print("Número de verdadeiros negativos: ", verdadeiros_negativos)
print("Porcentagem de verdadeiros negativos: {0:.3f}%".format(porcentagem_verdadeiros_negativos*100))

Número de verdadeiros negativos:  4
Porcentagem de verdadeiros negativos: 5.797%


In [32]:
falsos_negativos = 0
for i in range(len(lista_classificacao)):
    if lista_classificacao[i] == 4 and lista_resultado[i] == 0:
        falsos_negativos += 1
    elif lista_classificacao[i] == 4 and lista_resultado[i]== 1:
        falsos_negativos += 1
    elif lista_classificacao[i] == 3 and lista_resultado[i]== 0:
        falsos_negativos += 1
    elif lista_classificacao[i] == 3 and lista_resultado[i]== 1:
        falsos_negativos += 1
    elif lista_classificacao[i] == 2 and lista_resultado[i]== 0:
        falsos_negativos += 1
    elif lista_classificacao[i] == 2 and lista_resultado[i]== 1:
        falsos_negativos += 1
porcentagem_falsos_negativos = falsos_negativos/(len(test_relevante) + len(test_mrelevante))
print("Número de falsos negativos: ", falsos_negativos)
print("Porcentagem de falsos negativos: {0:.3f}%".format(porcentagem_falsos_negativos*100))

Número de falsos negativos:  44
Porcentagem de falsos negativos: 51.765%


___
### Concluindo

In [33]:
quadro = pd.crosstab(test['class'], test['Resultado'], margins = True, normalize = True)
quadro

Resultado,1,3,4,All
class,,,,
0,0.096045,0.112994,0.101695,0.310734
1,0.022599,0.056497,0.000000,0.079096
2,0.101695,0.028249,0.000000,0.129944
3,0.090395,0.101695,0.005650,0.197740
4,0.056497,0.225989,0.000000,0.282486
All,0.367232,0.525424,0.107345,1.000000


Não foi possível adicionar as classificações Muito irrelevante e Neutro na tabela de conclusão por erro no código.

O classificador não resultou em boa performance pois a porcentagem de verdadeiros positivos e negativos é de, respectivamente, 21.176% e 5.797%. Esses resultados demonstram uma alta taxa de erros na classificação produzida pelo classificador.

## DIferentes cenários:

- Pode ser usado para classificar textos ou artigos universitários
- Uso em pesquisa de mercado, para prever popularidade de algum produto
- Classificação de e-mails, para separar mensagens spam e relevantes


## Melhorias:
- Ampliar o quantidade de stopwords para obter uma filtragem melhor de palavras que não inferem sentimento no tweet.
- Aplicar uma função que detecte sarcasmo nas mensagens, melhorando a eficácia da classificação.
- Buscar erros no código, que levaram aos baixos valores de verdadeiros positivos e negativos.
- Rever a classificação realizada no excel antes do desenvolvimento do classificador.
- Simplificação do código para compactá-lo e demandar menos poder de computação.
- Aumentar a quantidade de tweets em geral para obtenção de mais amostras e treinar de forma mais ampla o classificador.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**